In [59]:
import torch
import torch.optim as optim
from torch.nn.functional import normalize
from torch import nn as nn
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import time
from torchvision import datasets, transforms
import torch.nn.functional as F
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Problem 1

In [60]:
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train = True, download = True)
cifar10_val = datasets.CIFAR10(data_path, train = False, download = True)
class_names = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

Files already downloaded and verified
Files already downloaded and verified


In [61]:
img, label = cifar10[100]
img, label, class_names[label]

(<PIL.Image.Image image mode=RGB size=32x32>, 8, 'ship')

In [62]:
tensor_cifar10 = datasets.CIFAR10(data_path, train = True, download = False, 
                                  transform = transforms.ToTensor())

In [63]:
imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3)
imgs_mean = imgs.view(3, -1).mean(dim = 1)
imgs_std = imgs.view(3, -1).std(dim = 1)
imgs_mean, imgs_std, imgs.shape

(tensor([0.4914, 0.4822, 0.4465]),
 tensor([0.2470, 0.2435, 0.2616]),
 torch.Size([3, 32, 32, 50000]))

In [64]:
normal_cifar10 = datasets.CIFAR10(data_path, train = True, download = False,
                                     transform = transforms.Compose([
                                         transforms.ToTensor(),
                                         transforms.Normalize(imgs_mean, imgs_std)
                                     ]))
normal_cifar10_val = datasets.CIFAR10(data_path, train = False, download = False,
                                     transform = transforms.Compose([
                                         transforms.ToTensor(),
                                         transforms.Normalize(imgs_mean, imgs_std)
                                     ]))

In [65]:
train_loader = torch.utils.data.DataLoader(normal_cifar10, batch_size = 100, shuffle = True)

model = nn.Sequential(nn.Linear(3072, 512),
                     nn.Tanh(),
                     nn.Linear(512, 10),
                     nn.LogSoftmax(dim=1))

loss_fn = nn.NLLLoss()
learning_rate = 1e-3
optimizer = optim.SGD(model.parameters(), lr = learning_rate)
n_epochs = 300

In [66]:
main_tic = time.perf_counter()
for epoch in range(1, n_epochs + 1):
    tic = time.perf_counter()
    loss_train = 0
    for imgs, labels in train_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_train += loss.item()
        
    toc = time.perf_counter()
    print(f"Epoch {epoch}, Duration = {round(toc - tic, 3)} seconds, Loss: {round(loss_train / len(train_loader), 5)}")
    
main_toc = time.perf_counter()
print(f"Total Training Time = {round(main_toc - main_tic, 3)} seconds")
print(f"Average Training Time per Epoch = {round((main_toc - main_tic) / n_epochs , 3)} seconds")

Epoch 1, Duration = 14.204 seconds, Loss: 2.04428
Epoch 2, Duration = 14.026 seconds, Loss: 1.90551
Epoch 3, Duration = 14.162 seconds, Loss: 1.85628
Epoch 4, Duration = 14.544 seconds, Loss: 1.82662
Epoch 5, Duration = 14.938 seconds, Loss: 1.80546
Epoch 6, Duration = 15.061 seconds, Loss: 1.78906
Epoch 7, Duration = 15.357 seconds, Loss: 1.77552
Epoch 8, Duration = 16.473 seconds, Loss: 1.76403
Epoch 9, Duration = 15.66 seconds, Loss: 1.75353
Epoch 10, Duration = 15.86 seconds, Loss: 1.7444
Epoch 11, Duration = 16.078 seconds, Loss: 1.73573
Epoch 12, Duration = 16.24 seconds, Loss: 1.72784
Epoch 13, Duration = 16.456 seconds, Loss: 1.72028
Epoch 14, Duration = 16.557 seconds, Loss: 1.71325
Epoch 15, Duration = 16.866 seconds, Loss: 1.70656
Epoch 16, Duration = 17.693 seconds, Loss: 1.70022
Epoch 17, Duration = 17.886 seconds, Loss: 1.694
Epoch 18, Duration = 21.058 seconds, Loss: 1.68814
Epoch 19, Duration = 18.841 seconds, Loss: 1.68239
Epoch 20, Duration = 17.311 seconds, Loss: 1.6

Epoch 162, Duration = 17.251 seconds, Loss: 1.22424
Epoch 163, Duration = 17.409 seconds, Loss: 1.22139
Epoch 164, Duration = 17.138 seconds, Loss: 1.21862
Epoch 165, Duration = 17.488 seconds, Loss: 1.2159
Epoch 166, Duration = 17.281 seconds, Loss: 1.21303
Epoch 167, Duration = 17.464 seconds, Loss: 1.21013
Epoch 168, Duration = 17.136 seconds, Loss: 1.20722
Epoch 169, Duration = 17.237 seconds, Loss: 1.20441
Epoch 170, Duration = 17.172 seconds, Loss: 1.20151
Epoch 171, Duration = 17.252 seconds, Loss: 1.19864
Epoch 172, Duration = 17.222 seconds, Loss: 1.19594
Epoch 173, Duration = 17.152 seconds, Loss: 1.19285
Epoch 174, Duration = 17.309 seconds, Loss: 1.19024
Epoch 175, Duration = 17.435 seconds, Loss: 1.1873
Epoch 176, Duration = 17.476 seconds, Loss: 1.18437
Epoch 177, Duration = 17.355 seconds, Loss: 1.18167
Epoch 178, Duration = 17.206 seconds, Loss: 1.17878
Epoch 179, Duration = 17.313 seconds, Loss: 1.17595
Epoch 180, Duration = 17.224 seconds, Loss: 1.17291
Epoch 181, Dur

In [67]:
val_loader = torch.utils.data.DataLoader(normal_cifar10_val, batch_size = 100, shuffle = False)

correct = 0
total = 0

val_tic = time.perf_counter()
with torch.no_grad():
    for imgs, labels in val_loader:
        batch_size = imgs.shape[0]
        outputs = model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim = 1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
val_toc = time.perf_counter()
print(f"Accuracy: {round(correct/total, 3)}, Duration = {round(val_toc - val_tic, 3)} seconds")

Accuracy: 0.495, Duration = 3.295 seconds


In [68]:
model_2 = nn.Sequential(nn.Linear(3072, 512),
                     nn.Tanh(),
                     nn.Linear(512, 128),
                     nn.Tanh(),
                     nn.Linear(128, 32),
                     nn.Tanh(),
                     nn.Linear(32, 10),
                     nn.LogSoftmax(dim=1))

optimizer_2 = optim.SGD(model_2.parameters(), lr = learning_rate)

main_tic_2 = time.perf_counter()
for epoch in range(1, n_epochs + 1):
    tic = time.perf_counter()
    loss_train = 0
    for imgs, labels in train_loader:
        batch_size = imgs.shape[0]
        outputs = model_2(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        optimizer_2.zero_grad()
        loss.backward()
        optimizer_2.step()
        loss_train += loss.item()
        
    toc = time.perf_counter()
    print(f"Epoch {epoch}, Duration = {round(toc - tic, 3)} seconds, Loss: {round(loss_train / len(train_loader), 5)}")
    
main_toc_2 = time.perf_counter()
print(f"Total Training Time = {round(main_toc_2 - main_tic_2, 3)} seconds")
print(f"Average Training Time per Epoch = ~{round((main_toc_2 - main_tic_2) / n_epochs , 3)} seconds")

Epoch 1, Duration = 19.577 seconds, Loss: 2.23611
Epoch 2, Duration = 17.588 seconds, Loss: 2.15574
Epoch 3, Duration = 17.607 seconds, Loss: 2.10332
Epoch 4, Duration = 18.112 seconds, Loss: 2.06216
Epoch 5, Duration = 18.03 seconds, Loss: 2.02852
Epoch 6, Duration = 17.574 seconds, Loss: 2.00065
Epoch 7, Duration = 17.825 seconds, Loss: 1.97726
Epoch 8, Duration = 17.605 seconds, Loss: 1.95717
Epoch 9, Duration = 20.02 seconds, Loss: 1.93946
Epoch 10, Duration = 20.368 seconds, Loss: 1.92345
Epoch 11, Duration = 18.185 seconds, Loss: 1.90866
Epoch 12, Duration = 17.715 seconds, Loss: 1.89485
Epoch 13, Duration = 17.721 seconds, Loss: 1.88186
Epoch 14, Duration = 17.736 seconds, Loss: 1.86972
Epoch 15, Duration = 17.759 seconds, Loss: 1.85827
Epoch 16, Duration = 17.704 seconds, Loss: 1.84747
Epoch 17, Duration = 17.879 seconds, Loss: 1.83738
Epoch 18, Duration = 17.656 seconds, Loss: 1.82778
Epoch 19, Duration = 17.735 seconds, Loss: 1.81875
Epoch 20, Duration = 17.973 seconds, Loss:

Epoch 162, Duration = 17.933 seconds, Loss: 1.15933
Epoch 163, Duration = 17.889 seconds, Loss: 1.15428
Epoch 164, Duration = 17.583 seconds, Loss: 1.14794
Epoch 165, Duration = 17.769 seconds, Loss: 1.142
Epoch 166, Duration = 17.943 seconds, Loss: 1.13645
Epoch 167, Duration = 17.825 seconds, Loss: 1.13045
Epoch 168, Duration = 17.909 seconds, Loss: 1.12407
Epoch 169, Duration = 17.681 seconds, Loss: 1.11804
Epoch 170, Duration = 17.531 seconds, Loss: 1.11198
Epoch 171, Duration = 17.737 seconds, Loss: 1.1061
Epoch 172, Duration = 17.806 seconds, Loss: 1.10002
Epoch 173, Duration = 17.939 seconds, Loss: 1.09369
Epoch 174, Duration = 17.895 seconds, Loss: 1.08798
Epoch 175, Duration = 18.011 seconds, Loss: 1.0819
Epoch 176, Duration = 17.913 seconds, Loss: 1.07575
Epoch 177, Duration = 17.738 seconds, Loss: 1.06906
Epoch 178, Duration = 17.726 seconds, Loss: 1.06415
Epoch 179, Duration = 17.609 seconds, Loss: 1.05718
Epoch 180, Duration = 17.672 seconds, Loss: 1.05141
Epoch 181, Durat

In [69]:
correct = 0
total = 0

val_tic_2 = time.perf_counter()
with torch.no_grad():
    for imgs, labels in val_loader:
        batch_size = imgs.shape[0]
        outputs = model_2(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim = 1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
val_toc_2 = time.perf_counter()
print(f"Accuracy: {round(correct/total, 3)}, Duration = {round(val_toc_2 - val_tic_2, 3)} seconds")

Accuracy: 0.458, Duration = 2.648 seconds


# Problem 2

In [70]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on the {device}.")

Training on the cuda.


In [71]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, padding = 1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size = 3, padding = 1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 8 * 8)
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

In [72]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    main_tic = time.perf_counter()
    for epoch in range(1, n_epochs + 1):
        loss_train = 0
        tic = time.perf_counter()
        for imgs, labels in train_loader:
            imgs = imgs.to(device = device)
            labels = labels.to(device = device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
            
        toc = time.perf_counter()
        print(f"Epoch {epoch}, Duration = {round(toc - tic, 3)} seconds, Loss: {round(loss_train / len(train_loader), 5)}")
    
    main_toc = time.perf_counter()
    print(f"Total Training Time = {round(main_toc - main_tic, 3)} seconds")
    print(f"Average Training Time per Epoch = {round((main_toc - main_tic) / n_epochs , 3)} seconds")

In [73]:
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        val_tic = time.perf_counter()
        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device = device)
                labels = labels.to(device = device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())
        val_toc = time.perf_counter()
            
        print(f"Loader: {name}, Accuracy: {round(correct/total, 3)}, Duration = {round(val_toc - val_tic, 3)} seconds")

In [74]:
model_cnn = Net()
if torch.cuda.is_available():
    model_cnn.cuda()
optimizer_cnn = optim.SGD(model_cnn.parameters(), lr = learning_rate)
loss_fn_cnn = nn.CrossEntropyLoss()

training_loop(n_epochs, optimizer_cnn, model_cnn, loss_fn_cnn, train_loader)

Epoch 1, Duration = 12.997 seconds, Loss: 2.2851
Epoch 2, Duration = 12.78 seconds, Loss: 2.23782
Epoch 3, Duration = 13.042 seconds, Loss: 2.1771
Epoch 4, Duration = 13.446 seconds, Loss: 2.12436
Epoch 5, Duration = 13.529 seconds, Loss: 2.08193
Epoch 6, Duration = 13.135 seconds, Loss: 2.04614
Epoch 7, Duration = 13.156 seconds, Loss: 2.01691
Epoch 8, Duration = 13.325 seconds, Loss: 1.99358
Epoch 9, Duration = 13.661 seconds, Loss: 1.97427
Epoch 10, Duration = 14.914 seconds, Loss: 1.95738
Epoch 11, Duration = 13.436 seconds, Loss: 1.94192
Epoch 12, Duration = 13.09 seconds, Loss: 1.92733
Epoch 13, Duration = 13.056 seconds, Loss: 1.91327
Epoch 14, Duration = 13.103 seconds, Loss: 1.89949
Epoch 15, Duration = 12.877 seconds, Loss: 1.88585
Epoch 16, Duration = 13.154 seconds, Loss: 1.87225
Epoch 17, Duration = 13.106 seconds, Loss: 1.85848
Epoch 18, Duration = 13.122 seconds, Loss: 1.84448
Epoch 19, Duration = 13.018 seconds, Loss: 1.83014
Epoch 20, Duration = 12.792 seconds, Loss: 1

Epoch 162, Duration = 13.098 seconds, Loss: 1.11255
Epoch 163, Duration = 13.092 seconds, Loss: 1.11048
Epoch 164, Duration = 13.484 seconds, Loss: 1.10885
Epoch 165, Duration = 13.062 seconds, Loss: 1.10686
Epoch 166, Duration = 13.035 seconds, Loss: 1.10504
Epoch 167, Duration = 16.761 seconds, Loss: 1.10335
Epoch 168, Duration = 15.938 seconds, Loss: 1.10127
Epoch 169, Duration = 14.261 seconds, Loss: 1.09972
Epoch 170, Duration = 14.901 seconds, Loss: 1.098
Epoch 171, Duration = 13.752 seconds, Loss: 1.0961
Epoch 172, Duration = 13.097 seconds, Loss: 1.09445
Epoch 173, Duration = 14.488 seconds, Loss: 1.09257
Epoch 174, Duration = 15.137 seconds, Loss: 1.09086
Epoch 175, Duration = 15.218 seconds, Loss: 1.08958
Epoch 176, Duration = 12.835 seconds, Loss: 1.0877
Epoch 177, Duration = 13.275 seconds, Loss: 1.08621
Epoch 178, Duration = 13.081 seconds, Loss: 1.08461
Epoch 179, Duration = 14.03 seconds, Loss: 1.08252
Epoch 180, Duration = 13.11 seconds, Loss: 1.08113
Epoch 181, Duratio

In [75]:
validate(model_cnn, train_loader, val_loader)

Loader: train, Accuracy: 0.674, Duration = 11.782 seconds
Loader: val, Accuracy: 0.655, Duration = 2.308 seconds


In [76]:
class Net_2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, padding = 1)
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size = 3, padding = 1)
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(8, 4, kernel_size = 3, padding = 1)
        self.act3 = nn.Tanh()
        self.pool3 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(4 * 4 * 4, 32)
        self.act4 = nn.Tanh()
        self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = self.pool3(self.act3(self.conv3(out)))
        out = out.view(-1, 4 * 4 * 4)
        out = self.act4(self.fc1(out))
        out = self.fc2(out)
        return out

In [77]:
model_cnn_2 = Net_2()
if torch.cuda.is_available():
    model_cnn_2.cuda()
optimizer_cnn_2 = optim.SGD(model_cnn_2.parameters(), lr = learning_rate)

training_loop(n_epochs, optimizer_cnn_2, model_cnn_2, loss_fn_cnn, train_loader)

Epoch 1, Duration = 14.527 seconds, Loss: 2.30895
Epoch 2, Duration = 14.062 seconds, Loss: 2.30442
Epoch 3, Duration = 14.324 seconds, Loss: 2.2992
Epoch 4, Duration = 13.531 seconds, Loss: 2.29186
Epoch 5, Duration = 13.458 seconds, Loss: 2.28079
Epoch 6, Duration = 13.471 seconds, Loss: 2.2647
Epoch 7, Duration = 13.448 seconds, Loss: 2.2437
Epoch 8, Duration = 13.433 seconds, Loss: 2.21953
Epoch 9, Duration = 13.572 seconds, Loss: 2.19428
Epoch 10, Duration = 13.49 seconds, Loss: 2.1692
Epoch 11, Duration = 13.483 seconds, Loss: 2.14481
Epoch 12, Duration = 13.29 seconds, Loss: 2.12185
Epoch 13, Duration = 13.508 seconds, Loss: 2.10172
Epoch 14, Duration = 13.487 seconds, Loss: 2.08554
Epoch 15, Duration = 13.42 seconds, Loss: 2.07293
Epoch 16, Duration = 13.522 seconds, Loss: 2.06283
Epoch 17, Duration = 13.44 seconds, Loss: 2.05426
Epoch 18, Duration = 13.507 seconds, Loss: 2.04654
Epoch 19, Duration = 13.249 seconds, Loss: 2.03923
Epoch 20, Duration = 14.876 seconds, Loss: 2.032

Epoch 162, Duration = 13.461 seconds, Loss: 1.35941
Epoch 163, Duration = 13.451 seconds, Loss: 1.35736
Epoch 164, Duration = 13.205 seconds, Loss: 1.35504
Epoch 165, Duration = 13.469 seconds, Loss: 1.35315
Epoch 166, Duration = 13.573 seconds, Loss: 1.3509
Epoch 167, Duration = 13.489 seconds, Loss: 1.3488
Epoch 168, Duration = 13.433 seconds, Loss: 1.34734
Epoch 169, Duration = 13.443 seconds, Loss: 1.34483
Epoch 170, Duration = 13.313 seconds, Loss: 1.34315
Epoch 171, Duration = 13.455 seconds, Loss: 1.34131
Epoch 172, Duration = 14.685 seconds, Loss: 1.33904
Epoch 173, Duration = 16.833 seconds, Loss: 1.33766
Epoch 174, Duration = 15.177 seconds, Loss: 1.33525
Epoch 175, Duration = 14.648 seconds, Loss: 1.33324
Epoch 176, Duration = 14.248 seconds, Loss: 1.33096
Epoch 177, Duration = 13.388 seconds, Loss: 1.32919
Epoch 178, Duration = 13.5 seconds, Loss: 1.3274
Epoch 179, Duration = 13.408 seconds, Loss: 1.32561
Epoch 180, Duration = 13.203 seconds, Loss: 1.32332
Epoch 181, Durati

In [78]:
validate(model_cnn_2, train_loader, val_loader)

Loader: train, Accuracy: 0.583, Duration = 10.645 seconds
Loader: val, Accuracy: 0.573, Duration = 2.113 seconds
